In [1]:
import os

In [2]:
path = r'D:\ML int\data_train_short\data_train_short'

In [3]:
list_path = list()
for dirpath, _, filenames in os.walk(path):
    for filename in filenames:
        if filename.startswith('-'):
            list_path.append(os.path.join(dirpath, filename))
    

In [4]:
from moviepy import VideoFileClip

In [5]:
clip = VideoFileClip(list_path[0])

# Вырезаем фрагмент с 10 по 20 секунду
cut_clip = clip.subclipped(10, 20)
cut_clip

In [5]:
import json
import pandas as pd
from datetime import datetime, timedelta

def time_to_seconds(t):
    h, m, s = map(int, t.split(":"))
    return h * 3600 + m * 60 + s

def fix_start_time_shift(t):
    """
    Сдвигает время на -1 минуту (если нужно исправить ошибку start > end)
    """
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        # Если формат некорректный, вернуть как есть
        return t
    dt -= timedelta(minutes=1)
    return dt.strftime("%H:%M:%S")

def load_and_clean_labels(json_path):
    """
    Загружает JSON с разметкой, возвращает корректный DataFrame с временными метками в секундах.
    Автоматически исправляет случаи, где start > end, уменьшая start на 1 минуту.
    """
    # Загрузка JSON
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Преобразуем в список словарей
    records = [
        {
            "id": key,
            "url": value["url"],
            "name": value["name"],
            "start": value["start"],
            "end": value["end"]
        }
        for key, value in data.items()
    ]

    # Создаем DataFrame
    df = pd.DataFrame(records)

    # Добавим колонки с временем в секундах
    df["start_sec"] = df["start"].apply(time_to_seconds)
    df["end_sec"] = df["end"].apply(time_to_seconds)

    # Исправим строки, где start > end — уменьшаем start на 1 минуту
    wrong_mask = df["start_sec"] > df["end_sec"]
    df.loc[wrong_mask, "start"] = df.loc[wrong_mask, "start"].apply(fix_start_time_shift)
    df.loc[wrong_mask, "start_sec"] = df.loc[wrong_mask, "start"].apply(time_to_seconds)

    return df

# Пример использования:
path_json = r'D:\ML int\labels_json\labels_json\train_labels.json'
data_train = load_and_clean_labels(path_json)


In [6]:
data_train

,id,url,name,start,end,start_sec,end_sec
0,-220020068_456255414,https://vkvideo.ru/video-220020068_456255414,"Баскетс. 4 сезон, 7 серия",0:01:42,0:01:47,102,107
1,-220020068_456249693,https://vkvideo.ru/video-220020068_456249693,"Бывaeт и xyжe. 1 сезон, 14 серия",0:00:06,0:00:10,6,10
2,-220020068_456255339,https://vkvideo.ru/video-220020068_456255339,Анатомия скандала. 1 сезон. 2 серия.,0:00:10,0:00:20,10,20
3,-220020068_456241755,https://vkvideo.ru/video-220020068_456241755,"Блеск. 2 сезон, 2 серия",0:03:08,0:03:12,188,192
4,-220020068_456241671,https://vkvideo.ru/video-220020068_456241671,"Амepикaнcкий вaндaл. 1 сезон, 3 серия",0:02:47,0:03:10,167,190
...,...,...,...,...,...,...,...
75,-220020068_456249719,https://vkvideo.ru/video-220020068_456249719,"Бывaeт и xyжe. 2 сезон ,6 серия",0:00:06,0:00:10,6,10
76,-220020068_456255400,https://vkvideo.ru/video-220020068_456255400,"Баскетс. 3 сезон, 3 серия",0:00:17,0:00:22,17,22
77,-220020068_456256446,https://vkvideo.ru/video-220020068_456256446,"Бойцовская ночь: Афера на миллион. 1 сезон, 4 ...",00:04:41,0:05:25,281,325
78,-220020068_456255401,https://vkvideo.ru/video-220020068_456255401,"Баскетс. 3 сезон, 4 серия",0:00:17,0:00:22,17,22


In [7]:
wrong_intervals = data_train["start_sec"] > data_train["end_sec"]

In [14]:
data_train[ data_train["start_sec"] > data_train["end_sec"]]

,id,url,name,start,end,start_sec,end_sec


In [9]:
data_train.loc[20, "end"] = "00:08:11"
data_train.loc[20, "end_sec"] = time_to_seconds("00:08:11")

In [10]:
data_train

,id,url,name,start,end,start_sec,end_sec
0,-220020068_456255414,https://vkvideo.ru/video-220020068_456255414,"Баскетс. 4 сезон, 7 серия",0:01:42,0:01:47,102,107
1,-220020068_456249693,https://vkvideo.ru/video-220020068_456249693,"Бывaeт и xyжe. 1 сезон, 14 серия",0:00:06,0:00:10,6,10
2,-220020068_456255339,https://vkvideo.ru/video-220020068_456255339,Анатомия скандала. 1 сезон. 2 серия.,0:00:10,0:00:20,10,20
3,-220020068_456241755,https://vkvideo.ru/video-220020068_456241755,"Блеск. 2 сезон, 2 серия",0:03:08,0:03:12,188,192
4,-220020068_456241671,https://vkvideo.ru/video-220020068_456241671,"Амepикaнcкий вaндaл. 1 сезон, 3 серия",0:02:47,0:03:10,167,190
...,...,...,...,...,...,...,...
75,-220020068_456249719,https://vkvideo.ru/video-220020068_456249719,"Бывaeт и xyжe. 2 сезон ,6 серия",0:00:06,0:00:10,6,10
76,-220020068_456255400,https://vkvideo.ru/video-220020068_456255400,"Баскетс. 3 сезон, 3 серия",0:00:17,0:00:22,17,22
77,-220020068_456256446,https://vkvideo.ru/video-220020068_456256446,"Бойцовская ночь: Афера на миллион. 1 сезон, 4 ...",00:04:41,0:05:25,281,325
78,-220020068_456255401,https://vkvideo.ru/video-220020068_456255401,"Баскетс. 3 сезон, 4 серия",0:00:17,0:00:22,17,22


In [13]:
data_train.loc[0, "end"] = "00:00:47"
data_train.loc[0, "end_sec"] = time_to_seconds("00:00:47")
data_train.loc[0, "start"] = "00:00:42"
data_train.loc[0, "start_sec"] = time_to_seconds("00:00:42")

In [12]:
def seconds_to_time(seconds):
    """
    Преобразует количество секунд в строку формата HH:MM:SS.
    """
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return f"{h:02}:{m:02}:{s:02}"

In [19]:
import os
import pandas as pd
from moviepy import VideoFileClip

def crop_intro_segments(df, video_paths, output_dir):
    """
    Обрезает видеофайлы по интро с расширением на 50% и сохраняет.
    
    Parameters:
    - df: DataFrame со столбцами 'id', 'start_sec', 'end_sec', 'name'
    - video_paths: список путей ко всем видеофайлам
    - output_dir: путь к папке, куда сохранять обрезки
    
    Returns:
    - Новый DataFrame с обновлёнными таймингами
    """
    os.makedirs(output_dir, exist_ok=True)
    video_index = {os.path.basename(p).split('.')[0]: p for p in video_paths}

    new_records = []

    for idx, row in df.iterrows():
        video_id = row['id']
        name = row['name']
        intro_start = row['start_sec']
        intro_end = row['end_sec']
        intro_duration = intro_end - intro_start

        if intro_duration <= 0:
            print(f"[!] Пропуск (некорректная длительность): {video_id}")
            continue

        file_path = video_index.get(video_id)
        if not file_path or not os.path.exists(file_path):
            print(f"[!] Видео не найдено: {video_id}")
            continue

        try:
            clip = VideoFileClip(file_path)
            video_duration = clip.duration

            shift = max(intro_duration, 5)
            new_start = max(0, intro_start - shift)
            new_end = intro_end + shift

            # Сохраняем видео
            output_path = os.path.join(output_dir, f"{video_id}.mp4")
            clip.subclipped(new_start, new_end).write_videofile(
                                output_path,
                                codec='libx264',
                                audio_codec='aac',
                                write_logfile=False,
                                logger=None  # если хочешь отключить вывод в консоль
                            )
            start_to_write = shift if new_start > 0 else new_start
            end_to_write = start_to_write + intro_duration  
            new_records.append({
                "id": video_id,
                "name": name,
                "original_start_sec": intro_start,
                "original_end_sec": intro_end,
                "new_start_sec": int(start_to_write),
                "new_end_sec": int(end_to_write),
                "new_start": seconds_to_time(int(new_start)),
                "new_end": seconds_to_time(int(new_end)),
                "video_duration": int(video_duration),
                "output_path": output_path
            })

        except Exception as e:
            print(f"[!] Ошибка с {video_id}: {e}")
            continue
    return pd.DataFrame(new_records)


In [20]:
new_df = crop_intro_segments(data_train, list_path, r'D:\ML int\data_train_short\cropped_v1')

In [21]:
new_df.to_csv(r'D:\ML int\data_train_short\cropped_v1\train.csv', encoding='utf-8-sig')

In [22]:
new_df

,id,name,original_start_sec,original_end_sec,new_start_sec,new_end_sec,new_start,new_end,video_duration,output_path
0,-220020068_456255414,"Баскетс. 4 сезон, 7 серия",42,47,5,10,00:00:37,00:00:52,1545,D:\ML int\data_train_short\cropped_v1\-2200200...
1,-220020068_456249693,"Бывaeт и xyжe. 1 сезон, 14 серия",6,10,5,9,00:00:01,00:00:15,1302,D:\ML int\data_train_short\cropped_v1\-2200200...
2,-220020068_456255339,Анатомия скандала. 1 сезон. 2 серия.,10,20,0,10,00:00:00,00:00:30,2695,D:\ML int\data_train_short\cropped_v1\-2200200...
3,-220020068_456241755,"Блеск. 2 сезон, 2 серия",188,192,5,9,00:03:03,00:03:17,1888,D:\ML int\data_train_short\cropped_v1\-2200200...
4,-220020068_456241671,"Амepикaнcкий вaндaл. 1 сезон, 3 серия",167,190,23,46,00:02:24,00:03:33,2219,D:\ML int\data_train_short\cropped_v1\-2200200...
...,...,...,...,...,...,...,...,...,...,...
75,-220020068_456249719,"Бывaeт и xyжe. 2 сезон ,6 серия",6,10,5,9,00:00:01,00:00:15,1250,D:\ML int\data_train_short\cropped_v1\-2200200...
76,-220020068_456255400,"Баскетс. 3 сезон, 3 серия",17,22,5,10,00:00:12,00:00:27,1439,D:\ML int\data_train_short\cropped_v1\-2200200...
77,-220020068_456256446,"Бойцовская ночь: Афера на миллион. 1 сезон, 4 ...",281,325,44,88,00:03:57,00:06:09,3057,D:\ML int\data_train_short\cropped_v1\-2200200...
78,-220020068_456255401,"Баскетс. 3 сезон, 4 серия",17,22,5,10,00:00:12,00:00:27,1371,D:\ML int\data_train_short\cropped_v1\-2200200...


In [35]:
#Для теста
path_test = r'D:\ML int\data_test_short\data_test_short'

In [36]:
test_list_path = list()
for dirpath, _, filenames in os.walk(path_test):
    for filename in filenames:
        if filename.startswith('-'):
            test_list_path.append(os.path.join(dirpath, filename))
    

In [37]:
import json
import pandas as pd
from datetime import datetime, timedelta

def time_to_seconds(t):
    h, m, s = map(int, t.split(":"))
    return h * 3600 + m * 60 + s

def fix_start_time_shift(t):
    """
    Сдвигает время на -1 минуту (если нужно исправить ошибку start > end)
    """
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        # Если формат некорректный, вернуть как есть
        return t
    dt -= timedelta(minutes=1)
    return dt.strftime("%H:%M:%S")

def load_and_clean_labels(json_path):
    """
    Загружает JSON с разметкой, возвращает корректный DataFrame с временными метками в секундах.
    Автоматически исправляет случаи, где start > end, уменьшая start на 1 минуту.
    """
    # Загрузка JSON
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Преобразуем в список словарей
    records = [
        {
            "id": key,
            "url": value["url"],
            "name": value["name"],
            "start": value["start"],
            "end": value["end"]
        }
        for key, value in data.items()
    ]

    # Создаем DataFrame
    df = pd.DataFrame(records)

    # Добавим колонки с временем в секундах
    df["start_sec"] = df["start"].apply(time_to_seconds)
    df["end_sec"] = df["end"].apply(time_to_seconds)

    # Исправим строки, где start > end — уменьшаем start на 1 минуту
    wrong_mask = df["start_sec"] > df["end_sec"]
    df.loc[wrong_mask, "start"] = df.loc[wrong_mask, "start"].apply(fix_start_time_shift)
    df.loc[wrong_mask, "start_sec"] = df.loc[wrong_mask, "start"].apply(time_to_seconds)

    return df

# Пример использования:
path_json = r'D:\ML int\labels_json\labels_json\test_labels.json'
data_test = load_and_clean_labels(path_json)


In [38]:
data_test

,id,url,name,start,end,start_sec,end_sec
0,-220020068_456249220,https://vkvideo.ru/video-220020068_456249220,"24 часа. 2 сезон, 16 серия",0:00:05,0:00:16,5,16
1,-220020068_456249373,https://vkvideo.ru/video-220020068_456249373,"24 часа. 8 сезон, 18 серия",0:00:05,0:00:16,5,16
2,-220020068_456249231,https://vkvideo.ru/video-220020068_456249231,"24 часа. 3 сезон, 1 серия",0:00:05,0:00:16,5,16
3,-220020068_456255339,https://vkvideo.ru/video-220020068_456255339,Анатомия скандала. 1 сезон. 2 серия.,0:00:10,0:00:20,10,20
4,-220020068_456249284,https://vkvideo.ru/video-220020068_456249284,"24 часа. 5 сезон, 4 серия",0:00:05,0:00:16,5,16
5,-220020068_456241671,https://vkvideo.ru/video-220020068_456241671,"Амepикaнcкий вaндaл. 1 сезон, 3 серия",00:02:47,0:03:10,167,190
6,-220020068_456249192,https://vkvideo.ru/video-220020068_456249192,"24 часа. 1 сезон, 11 серия",0:00:05,0:00:15,5,15
7,-220020068_456249257,https://vkvideo.ru/video-220020068_456249257,"24 часа. 4 сезон, 19 серия",0:00:05,0:00:15,5,15
8,-220020068_456249375,https://vkvideo.ru/video-220020068_456249375,"24 часа. 8 сезон, 20 серия",0:00:05,0:00:15,5,15
9,-220020068_456256475,https://vkvideo.ru/video-220020068_456256475,"Бойцовская ночь: Афера на миллион. 1 сезон, 5 ...",00:09:42,0:10:25,582,625


In [39]:
data_test[ data_test["start_sec"] > data_test["end_sec"]]

,id,url,name,start,end,start_sec,end_sec


In [40]:
new_df_test = crop_intro_segments(data_test, test_list_path, r'D:\ML int\data_test_short\cropped_v1')

[!] Ошибка с -220020068_456256475: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: '-220020068_456256475TEMP_MPY_wvf_snd.mp4'


In [42]:
new_df_test.to_csv(r'D:\ML int\data_test_short\cropped_v1\test.csv', encoding='utf-8-sig')

In [41]:
new_df_test

,id,name,original_start_sec,original_end_sec,new_start_sec,new_end_sec,new_start,new_end,video_duration,output_path
0,-220020068_456249220,"24 часа. 2 сезон, 16 серия",5,16,0,11,00:00:00,00:00:27,2638,D:\ML int\data_test_short\cropped_v1\-22002006...
1,-220020068_456249373,"24 часа. 8 сезон, 18 серия",5,16,0,11,00:00:00,00:00:27,2642,D:\ML int\data_test_short\cropped_v1\-22002006...
2,-220020068_456249231,"24 часа. 3 сезон, 1 серия",5,16,0,11,00:00:00,00:00:27,3107,D:\ML int\data_test_short\cropped_v1\-22002006...
3,-220020068_456255339,Анатомия скандала. 1 сезон. 2 серия.,10,20,0,10,00:00:00,00:00:30,2695,D:\ML int\data_test_short\cropped_v1\-22002006...
4,-220020068_456249284,"24 часа. 5 сезон, 4 серия",5,16,0,11,00:00:00,00:00:27,2629,D:\ML int\data_test_short\cropped_v1\-22002006...
5,-220020068_456241671,"Амepикaнcкий вaндaл. 1 сезон, 3 серия",167,190,23,46,00:02:24,00:03:33,2219,D:\ML int\data_test_short\cropped_v1\-22002006...
6,-220020068_456249192,"24 часа. 1 сезон, 11 серия",5,15,0,10,00:00:00,00:00:25,2565,D:\ML int\data_test_short\cropped_v1\-22002006...
7,-220020068_456249257,"24 часа. 4 сезон, 19 серия",5,15,0,10,00:00:00,00:00:25,2645,D:\ML int\data_test_short\cropped_v1\-22002006...
8,-220020068_456249375,"24 часа. 8 сезон, 20 серия",5,15,0,10,00:00:00,00:00:25,2582,D:\ML int\data_test_short\cropped_v1\-22002006...
9,-220020068_456249206,"24 часа. 2 сезон, 2 серия",5,16,0,11,00:00:00,00:00:27,2627,D:\ML int\data_test_short\cropped_v1\-22002006...


In [34]:
len(test_list_path)

80